> Javascript is an object-based language, not a class-based language.

If you know Java or C++, you should be familiar with the inheritance concept. **In this programming paradigm**, a class is a blueprint for creating objects. If you want a new class to reuse the functionality of an existing class, you can create a new class that extends the existing class. This is called **classical inheritance**.

> JavaScript doesn’t use **classical inheritance**. Instead, it uses **prototypal inheritance**.

In prototypal inheritance, an object “inherits” properties from another object via the `prototype` linkage.

---

## 1. JavaScript prototypal inheritance and `__proto__`

在`Mynote/javascript/prototype chain`那一节, 对prototype chain做了个简单的介绍, 其实prototype chain的意义正是代替了传统语言java,cpp这种类的继承机制, 只不过是对象之间的继承, 而不是类之间. 

```js
const person = {
    name: 'Jack',
    greet: function () {
        console.log('hello, from ' + this.name);
    }
};

const student = {
    study: function () {
        return 'study';
    }
}
student.__proto__ = person;

// console.log(student.name); //Jack
// console.log(student.age); //error
student.name = 'John';
student.greet(); //hello, from John

console.log(student.__proto__); //{ name: 'Jack', greet: [Function: greet] }
console.log(Object.getPrototypeOf(student)); //{ name: 'Jack', greet: [Function: greet] }
```

官方说`obj.__proto__`不是访问对象prototype属性的标准方法,因为有的browser可能没实现, 所以访问(只读)一般用`Object.getPrototypeOf(student)`这种方式去做, 那我们想赋值给一个对象的prototype要怎么做呢? 答: 使用`Object.create(fatherObject)`去创造一个子对象. 例子(假如对象person还是上面的对象):
```js
// 创造新的子对象
const student = Object.create(person);
student.name = 'John';
student.study = function () {
    console.log('studying');
}

student.greet(); //hello, from John
student.study(); //studying
console.log(Object.getPrototypeOf(student)); //{ name: 'Jack', greet: [Function: greet] }
```

---

## 2. Javascript 不存在真正的类

看了上面的解释, 相信也对js语言本身有了基本的了解, js不存在真正的类, 即使是ES6之后, 类也不过是个语法糖(syntactic sugar)而已. 不信? 事实说话:

```js
class Person {
    constructor(name) {
        this.name = name;
    }
    getName() {
        return this.name;
    }
}

console.log(typeof Person); //function
```

---

> Unlike other programming languages such as Java and C#, JavaScript classes are syntactic sugar over the **prototypal inheritance**. In other words, ES6 classes are just special functions. 但是 class declarations are not **hoisted** like function declarations.

`Mynote/javascript/prototype chain`中提到的`obj.prototype`和`obj.__proto__`不是一回事, 当定义一个普通的对象之后(不是函数)它们的`obj.prototype`是`undefined`, 而当定义一个函数之后, 这个函数的`func.prototype`会被自动创建, 并且值为一个空对象`{}`:

```js
const func = function () {
    this.name = 'John';
}
const cap = {
    size: 36,
    price: '30CAD',
}

console.log(func.prototype); //{}
console.log(cap.prototype); //undefined
```

---

## 3. 使用函数创建新的对象背后发生的事

考虑下面一段代码, 想一下都是发生了什么:
```js
function Person(name) {
    this.name = name;
}
Person.prototype.getName = function () {
    return this.name;
};

console.log(Person.prototype); //{ getName: [Function (anonymous)] }

const john = new Person("John Doe");
console.log(Object.getPrototypeOf(john)); //{ getName: [Function (anonymous)] }

console.log(john); //Person { name: 'John Doe' }
console.log(john.getName()); //John Doe
```
第一步发生在定义函数Person后:

- 1. `Person.prototype`被创建, 并被赋值为一个空对象`{}`

- 2. 给`Person.prototype`对象增加一个函数, 名为`getName`

此时`Person.prototype`不再是个空对象, 而是有一个名为`getName`函数的对象`{ getName: [Function (anonymous)] }`

之后三步发生在调用`new`关键字后:

- 3. 新创建空对象`{}`并赋值给john
- 4. 执行`john.__proto__ = Person.prototype`
- 5. 执行函数Person, 函数体里的`this.name = name`变为`john.name = name`

> 综上总结一句话: The `john` object is an instance of the `Person` through **prototypal inheritance**. 

https://www.javascripttutorial.net/es6/javascript-class/